In [252]:
import pandas as pd
from data_retrievel_and_feature_extraction import uniprot_info as uni
from Bio.PDB import PDBParser

In [253]:
# Gene
gene = "TECTA"
gene_len = uni.get_sequence_length(gene)

# File names
prediction_vs_real_path = "C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\globular_proteins\\predictions_vs_real_globular_proteins.csv"
pathogenic_b_factor_output_path = f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\globular_proteins\\b_factor\\{gene}_pathogenic_b_factor_output_globular_proteins.csv"
benign_b_factor_output_path = f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\globular_proteins\\b_factor\\{gene}_benign_b_factor_output_globular_proteins.csv"
pdb_file_path = f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\globular_proteins\\pdb_files\\{gene}.pdb"

# Load the predictions_vs_real data
predictions_vs_real_df = pd.read_csv(prediction_vs_real_path)
predictions_vs_real_df


,gene,variant,prediction,reality
0,POLR1D,L111P,1,1
1,POLR1D,T50I,1,1
2,POLR1D,L55V,1,1
3,POLR1D,G99S,1,1
4,POLR1D,C68W,1,1
...,...,...,...,...
2306,SERPINB6,V346I,0,0
2307,SERPINB6,G153S,1,0
2308,SERPINB6,M90V,1,0
2309,BDP1,V275I,0,0


In [254]:
# Extract the data for the gene
gene_predictions = predictions_vs_real_df[predictions_vs_real_df["gene"] == gene]

# Add column for position of the variant, according to the "variant" column, which is in the format "L111P"
gene_predictions["position"] = gene_predictions["variant"].apply(lambda x: int(x[1:-1]))

# Add column that contains TP/FP/FN/TN according to the prediction and the reality columns.
gene_predictions["TP/FP/FN/TN"] = gene_predictions.apply(lambda x: "TP" if x["prediction"] == 1 and x["reality"] == 1 else "FP" if x["prediction"] == 1 and x["reality"] == 0 else "FN" if x["prediction"] == 0 and x["reality"] == 1 else "TN", axis=1)

gene_prediction_pathogenic = gene_predictions[gene_predictions["reality"] == 1]
gene_prediction_benign = gene_predictions[gene_predictions["reality"] == 0]

C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_22692\708265304.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_predictions["position"] = gene_predictions["variant"].apply(lambda x: int(x[1:-1]))
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_22692\708265304.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_predictions["TP/FP/FN/TN"] = gene_predictions.apply(lambda x: "TP" if x["prediction"] == 1 and x["reality"] == 1 else "FP" if x["prediction"] == 1 and x["reality"] == 0 else "FN" if x[

In [255]:
"""
Create 2 dataframes (benign/pathogenic) that contains values for each position in the gene, according to the TP/FP/FN/TN column. The values will be determined in this way:
For pathogenic variants:
- TP: 1
- FN: 0
- No data, : 3
For benign variants:
- TN: 1
- FP: 0
- No data: 3
# """
benign_b_factor = pd.DataFrame(columns=["position", "b_factor"])
pathogenic_b_factor = pd.DataFrame(columns=["position", "b_factor"])

# For each position in the gene, check if there is data for this position, and insert the value to the right dataframe.
for position in range(1, gene_len+1):
    # Check if there is data for this position in the gene_prediction_pathogenic dataframe
    if position in gene_prediction_pathogenic["position"].values:
        # Check how many instances of this position are there
        position_data = gene_prediction_pathogenic[gene_prediction_pathogenic["position"] == position]
        # If there is only one instance, take the value of the TP/FP/FN/TN column
        if len(position_data) == 1:
            if position_data["TP/FP/FN/TN"].values[0] == "TP":
                # Insert the position and the b_factor value to the pathogenic_b_factor dataframe, without "append" method
                pathogenic_b_factor.loc[pathogenic_b_factor.shape[0]] = [position, 1]
            elif position_data["TP/FP/FN/TN"].values[0] == "FN":
                pathogenic_b_factor.loc[pathogenic_b_factor.shape[0]] = [position, 0]
        # If there is more than one instance, check if the predictions are the same, if so- take this value of the TP/FP/FN/TN column. If they are not the same, append "2".
        else:
            # if all the values are the same, take the value of the TP/FP/FN/TN column, and for the other dataframe insert "3" (no data)
            if len(position_data["TP/FP/FN/TN"].unique()) == 1:
                if position_data["TP/FP/FN/TN"].values[0] == "TP":
                    pathogenic_b_factor.loc[pathogenic_b_factor.shape[0]] = [position, 1]
                elif position_data["TP/FP/FN/TN"].values[0] == "FN":
                    pathogenic_b_factor.loc[pathogenic_b_factor.shape[0]] = [position, 0]
            # If the values are not the same, insert "2" for mulitple values
            else:
                pathogenic_b_factor.loc[pathogenic_b_factor.shape[0]] = [position, 2]       
    # If there is no data for this position, insert "3" for no data
    else:
        pathogenic_b_factor.loc[pathogenic_b_factor.shape[0]] = [position, 3]
        
# For each position in the gene, check if there is data for this position, and insert the value to the right dataframe.
for position in range(1, gene_len+1):
    # Check if there is data for this position in the gene_prediction_pathogenic dataframe
    if position in gene_prediction_benign["position"].values:
        # Check how many instances of this position are there
        position_data = gene_prediction_benign[gene_prediction_benign["position"] == position]
        # If there is only one instance, take the value of the TP/FP/FN/TN column
        if len(position_data) == 1:
            if position_data["TP/FP/FN/TN"].values[0] == "TN":
                # Insert the position and the b_factor value to the pathogenic_b_factor dataframe, without "append" method
                benign_b_factor.loc[benign_b_factor.shape[0]] = [position, 1]
            elif position_data["TP/FP/FN/TN"].values[0] == "FP":
                benign_b_factor.loc[benign_b_factor.shape[0]] = [position, 0]
        # If there is more than one instance, check if the predictions are the same, if so- take this value of the TP/FP/FN/TN column. If they are not the same, append "2".
        else:
            # if all the values are the same, take the value of the TP/FP/FN/TN column, and for the other dataframe insert "3" (no data)
            if len(position_data["TP/FP/FN/TN"].unique()) == 1:
                if position_data["TP/FP/FN/TN"].values[0] == "TN":
                    benign_b_factor.loc[benign_b_factor.shape[0]] = [position, 1]
                elif position_data["TP/FP/FN/TN"].values[0] == "FP":
                    benign_b_factor.loc[benign_b_factor.shape[0]] = [position, 0]
            # If the values are not the same, insert "2" for mulitple values
            else:
                benign_b_factor.loc[benign_b_factor.shape[0]] = [position, 2]       
    # If there is no data for this position, insert "3" for no data
    else:
        benign_b_factor.loc[benign_b_factor.shape[0]] = [position, 3]

In [256]:
# Save the b_factor columns to csv files, without the index column and position column
pathogenic_b_factor

,position,b_factor
0,1,3
1,2,3
2,3,3
3,4,3
4,5,3
...,...,...
2150,2151,3
2151,2152,3
2152,2153,3
2153,2154,3


In [257]:
benign_b_factor

,position,b_factor
0,1,3
1,2,3
2,3,3
3,4,3
4,5,3
...,...,...
2150,2151,3
2151,2152,3
2152,2153,1
2153,2154,3


In [258]:

def get_bfactor_column_for_pdb_file(pdb_file_path, b_factor_per_position):
    """
    Create a series that contains the b_factor values for each atom in the pdb file, according to the position of the atom in the pdb file.
    :param pdb_file_path: The path to the pdb file
    :param b_factor_per_position: The b_factor values for each position in the gene
    :return: A series that contains the b_factor values for each atom in the pdb file, according to the position of the atom in the pdb file.
    """
    # Create a DataFrame from the pdb file
    pdb_df = pdb_to_dataframe(pdb_file_path)
    # Create a series that contains the b_factor values for each atom in the pdb file, according to the position of the atom in the pdb file.
    b_factor_series = pd.Series(index=range(1, len(pdb_df) + 1), name="b_factor")
    for index, row in pdb_df.iterrows():
        # Get the position of the atom in the gene
        position = row["RESIDUESEQ"]
        # Get the b_factor value for this position
        b_factor = b_factor_per_position[b_factor_per_position["position"] == position]["b_factor"].values[0]
        # Insert the b_factor value to the series
        b_factor_series[index+1] = b_factor
    return b_factor_series


def pdb_to_dataframe(pdb_file_path):
    # Create a PDB parser and load the structure
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file_path)

    # Initialize lists to store PDB data
    atom_data = []

    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    atom_record = {
                        'ATOM': atom.get_serial_number(),
                        'NAME': atom.get_name(),
                        'RESNAME': residue.get_resname(),
                        'CHAINID': chain.id,
                        'RESIDUESEQ': residue.id[1],
                        'X': atom.get_coord()[0],
                        'Y': atom.get_coord()[1],
                        'Z': atom.get_coord()[2],
                        'OCCUPANCY': atom.get_occupancy(),
                        'TEMPFACTOR': atom.get_bfactor(),
                        'ELEMENT': atom.element,
                    }
                    atom_data.append(atom_record)

    # Create a DataFrame from the atom data
    df = pd.DataFrame(atom_data)

    return df


In [259]:

pathogenic_b_factor_by_pdb = get_bfactor_column_for_pdb_file(pdb_file_path, pathogenic_b_factor)
benign_b_factor_by_pdb = get_bfactor_column_for_pdb_file(pdb_file_path, benign_b_factor)

# Multiply the b_factor values by 10, to get the same scale as the b_factor values in the pdb file
pathogenic_b_factor_by_pdb = pathogenic_b_factor_by_pdb * 10
benign_b_factor_by_pdb = benign_b_factor_by_pdb * 10
pathogenic_b_factor_by_pdb

1        30.0
2        30.0
3        30.0
4        30.0
5        30.0
         ... 
16767    30.0
16768    30.0
16769    30.0
16770    30.0
16771    30.0
Name: b_factor, Length: 16771, dtype: float64

In [260]:
# Save the b_factor columns to csv files, without the index column
pathogenic_b_factor_by_pdb.to_csv(pathogenic_b_factor_output_path, index=True, header=True)
benign_b_factor_by_pdb.to_csv(benign_b_factor_output_path, index=True, header=True)